Sekcja 11 - More Text Processing With Regular Expressions

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


More text processing with Regular Expressions

In [ ]:
%%bigquery --project august-bucksaw-330517
select occurrence_range, regexp_extract(occurrence_range, r'\d') extracted
from `jrjames83-1171.sampledata.aoc2017day2`
limit 5

,occurrence_range,extracted
0,6-12,6
1,9-16,9
2,10-15,1
3,6-13,6
4,6-15,6


In [ ]:
%%bigquery --project august-bucksaw-330517
select occurrence_range, 
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-\d+') high_char,
from jrjames83-1171.sampledata.aoc2017day2
limit 5

,occurrence_range,low_char,high_char
0,6-12,6,-12
1,9-16,9,-16
2,10-15,10,-15
3,6-13,6,-13
4,6-15,6,-15


In [ ]:
%%bigquery --project august-bucksaw-330517
select *, occurrence_range, 
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-\d+') high_char,
regexp_extract_all(datafield, 'b') matching_bs
from jrjames83-1171.sampledata.aoc2017day2
limit 5

,occurrence_range,character,datafield,occurrence_range_1,low_char,high_char,matching_bs
0,6-12,b,wbfgbghzbmvf,6-12,6,-12,"[b, b, b]"
1,9-16,b,bbbbbbbbkbbbbbbbbbbb,9-16,9,-16,"[b, b, b, b, b, b, b, b, b, b, b, b, b, b, b, ..."
2,10-15,b,bbbdbmqbkjbbtbd,10-15,10,-15,"[b, b, b, b, b, b, b, b]"
3,6-13,b,qflzbbbcfzkbmtx,6-13,6,-13,"[b, b, b, b]"
4,6-15,b,kptphpbxxqsbxlhxplml,6-15,6,-15,"[b, b]"


In [ ]:
%%bigquery --project august-bucksaw-330517
select *, occurrence_range, 
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-\d+') high_char,
array_length(regexp_extract_all(datafield, character)) nbr_matches
from jrjames83-1171.sampledata.aoc2017day2
limit 5

,occurrence_range,character,datafield,occurrence_range_1,low_char,high_char,nbr_matches
0,6-12,b,wbfgbghzbmvf,6-12,6,-12,3
1,9-16,b,bbbbbbbbkbbbbbbbbbbb,9-16,9,-16,19
2,10-15,b,bbbdbmqbkjbbtbd,10-15,10,-15,8
3,6-13,b,qflzbbbcfzkbmtx,6-13,6,-13,4
4,6-15,b,kptphpbxxqsbxlhxplml,6-15,6,-15,2


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select *, occurrence_range, 
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-\d+') high_char,
array_length(regexp_extract_all(datafield, character)) nbr_matches
from jrjames83-1171.sampledata.aoc2017day2)

select * from (
    select *,
    nbr_matches between cast(low_char as int64) and cast(high_char as int64) as outcome
    from base_table)
where outcome is false
limit 5

,occurrence_range,character,datafield,occurrence_range_1,low_char,high_char,nbr_matches,outcome
0,6-12,b,wbfgbghzbmvf,6-12,6,-12,3,False
1,9-16,b,bbbbbbbbkbbbbbbbbbbb,9-16,9,-16,19,False
2,10-15,b,bbbdbmqbkjbbtbd,10-15,10,-15,8,False
3,6-13,b,qflzbbbcfzkbmtx,6-13,6,-13,4,False
4,6-15,b,kptphpbxxqsbxlhxplml,6-15,6,-15,2,False


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as 
(select *,  
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-(\d+)') high_char
from jrjames83-1171.sampledata.aoc2017day2),

second_table as (
select *, 
substr(datafield, cast(low_char as int64), 1) datafield_at_low,
substr(datafield, cast(high_char as int64), 1) datafield_at_high
from base_table)

select * from (
    select *, 
    case when datafield_at_low = character AND datafield_at_high = character then false
    when datafield_at_low = character or datafield_at_high = character then True
    else False end outcome
-- case 1, datafield_at_low = character OR datafield_at_high = character (one equal)
-- case 2, datafield_at_low != character AND datafield_at_high != character (neither equal)
-- case 3, datafield_at_low = character AND datafield_at_high = character (both equal)
from second_table 
)
where outcome is true
limit 10

,occurrence_range,character,datafield,low_char,high_char,datafield_at_low,datafield_at_high,outcome
0,9-16,b,bbbbbbbbkbbbbbbbbbbb,9,16,k,b,True
1,6-13,b,qflzbbbcfzkbmtx,6,13,b,m,True
2,9-14,b,bbbsbqbbjjbbkb,9,14,j,b,True
3,14-19,c,ccpczqrccfhtcvljlvc,14,19,v,c,True
4,13-18,c,ccccccccccccdccccccc,13,18,d,c,True
5,12-17,c,hcctccccbqcctscsscv,12,17,c,s,True
6,11-16,c,plvcnjcpnkvcpcmcpbc,11,16,v,c,True
7,6-15,d,ddddddgddpddddwd,6,15,d,w,True
8,5-12,d,pzdrdqxzqzwgzwcgsv,5,12,d,g,True
9,11-18,f,fgffgwlffffftfzfrt,11,18,f,t,True


SQL UDF (JS)

https://cloud.google.com/bigquery/docs/reference/standard-sql/user-defined-functions

In [ ]:
%%bigquery --project august-bucksaw-330517

create temp function    chooseOutcome(datafield_at_low string, 
                        datafield_at_high string, 
                        target_char string)
returns bool 
language js as """
    if (datafield_at_low == target_char && datafield_at_high == target_char) {
        return false
    } else if (datafield_at_low == target_char || datafield_at_high == target_char) {
        return true
    } else {
        return false
    }
""";

with base_table as 
(select *,  
regexp_extract(occurrence_range, r'\d+') low_char,
regexp_extract(occurrence_range, r'-(\d+)') high_char
from jrjames83-1171.sampledata.aoc2017day2),

second_table as (
select *, 
substr(datafield, cast(low_char as int64), 1) datafield_at_low,
substr(datafield, cast(high_char as int64), 1) datafield_at_high
from base_table)

select * from (
select *, chooseOutcome(datafield_at_low, datafield_at_high, character) as outcome
from second_table   
) where outcome is true
limit 10

,occurrence_range,character,datafield,low_char,high_char,datafield_at_low,datafield_at_high,outcome
0,9-16,b,bbbbbbbbkbbbbbbbbbbb,9,16,k,b,True
1,6-13,b,qflzbbbcfzkbmtx,6,13,b,m,True
2,9-14,b,bbbsbqbbjjbbkb,9,14,j,b,True
3,14-19,c,ccpczqrccfhtcvljlvc,14,19,v,c,True
4,13-18,c,ccccccccccccdccccccc,13,18,d,c,True
5,12-17,c,hcctccccbqcctscsscv,12,17,c,s,True
6,11-16,c,plvcnjcpnkvcpcmcpbc,11,16,v,c,True
7,6-15,d,ddddddgddpddddwd,6,15,d,w,True
8,5-12,d,pzdrdqxzqzwgzwcgsv,5,12,d,g,True
9,11-18,f,fgffgwlffffftfzfrt,11,18,f,t,True
